# RNN

## 例題
sin波をRNN（BPTT1）で予測するプログラムを作成せよ．

### STEP1：インポート
下記ライブラリをインポートする．

- torch
- torch.nn
- torch.nn.functional
- torch.optim
- torchvision
- torchvision.transforms
- numpy
- matplotlib

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

### STEP2：ネットワーク定義
ネットワークを定義するクラスを記述する． 

- クラスはnn.Moduleを継承する
- クラスは```__init__()```と```forward()```というメソッドを定義する
    - ```__init__()```：ネットワークの構造を定義
    - ```forward()```：ネットワークの順伝播を定義
- RNN層は```nn.RNN```クラスを用いる
  - RNN層には活性化関数（デフォルト：tanh）が組み込まれている
- 全結合層は```nn.Linear```クラスを用いる


In [ ]:
# 任意のクラス名（ここではRNN_BPTT1）をつける
class RNN_BPTT1(nn.Module):

    # ネットワークの構造を定義
    def __init__(self):
        super(RNN_BPTT1, self).__init__()

        # RNN層の定義
        # 引数1：入力サイズ = 1
        # 引数2：出力サイズ = 32
        self.rnn = nn.RNN(1, 32)

        # 全結合層の定義
        # 引数1：入力サイズ = 32
        # 引数2：出力サイズ = 1
        self.fc = nn.Linear(32, 1)


    # ネットワークの順伝播を定義
    def forward(self, x):
        # 引数xは(入力データの時間の長さ, バッチサイズ, 入力サイズ)の形式

        # RNN層の順伝播
        # 活性化関数（tanh）が組み込まれている
        # 引数：入力データ
        # 返り値1：出力データ
        # 返り値2：隠れ層の状態
        x, _ = self.rnn(x)

        # 全結合層の順伝播
        x = self.fc(x)

        # 返り値xは(入力データの時間の長さ, バッチサイズ, 出力サイズ)の形式
        return x

In [ ]:
# 上記RNN_BPTT1クラスのインスタンスを生成
rnn_bptt1 = RNN_BPTT1()

# 実行するデバイスにインスタンスを移動
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
rnn_bptt1.to(device)

### STEP3：誤差関数・最適化器
誤差関数と最適化器を定義する．
- 誤差関数：```nn.MSELoss```と```nn.CrossEntropyLoss```がよく使われる
    - ```nn.MSELoss```：回帰問題でよく使う（今回はこちらを使う）
    - ```nn.CrossEntropyLoss```：分類問題でよく使う
- 最適化器：```optim.SGD```と```optim.Adam```がよく使われる
    - ```optim.SGD```：確率的勾配降下法（今回はこちらを使う）
    - ```optim.Adam```：Adam

In [ ]:
# 誤差関数（平均二乗誤差）の定義
criterion = nn.MSELoss()

# 最適化器（確率的勾配降下法）の定義
# 引数1：最適化するネットワークのパラメータ parameters()で取得可能
# 引数2：学習率 = 0.05
optimizer = optim.SGD(rnn_bptt1.parameters(), lr=0.05)

### STEP4：データセット
データセットとなるsin波を用意する．
- Training用データとして1周期100点となるsin波を部分的に切り取って複数の初期位相をもつsin波を用意する
  - train_in：Training用，入力データ
  - train_ta：Training用，教師データ
  - 配列の形状：(時間の長さ, 総データ数, 入力サイズ) = (175, 25, 1)
  - train_taはtrain_inより1時刻先のデータ
  - train_inが入力された時train_taを予測するように学習
- Test用データとして1周期100点となるcos波を部分的に切り取って複数の初期位相をもつcos波を用意する
  - test_in：Test用，入力データ
  - test_ta：Test用，正解データ
  - 配列の形状：(時間の長さ, 総データ数, 入力サイズ) = (75, 25, 1)
  - test_taはtest_inより1時刻先のデータ
  - test_inが入力された時test_taを予測できるかテスト

In [ ]:
# 総データ数
D_SIZE = 25

# sin波を生成
# 引数1：sin波1周期のデータ点数
# 引数2：sin波の周期
# 引数3：sin波の初期位相
def gen_sin(num_div=100, cycles=2, offset=0):
    step = 2 * np.pi / num_div
    data = [np.sin(step * i + offset) for i in range(num_div * cycles + 1)]
    return data

# 入力データと教師（正解）データの生成
# 引数1：sin波1周期のデータ点数
# 引数2：sin波の周期
# 引数3：sin波の個数
# 引数4：sin波の初期位相
def make_train_data(num_div=100, cycles=2, batch_size=25, offset=0):
    sin_wave = gen_sin(num_div, cycles, offset)
    count = len(sin_wave) - batch_size

    in_data = [sin_wave[idx:idx+batch_size] for idx in range(count)]
    in_data = torch.tensor(in_data, dtype=torch.float)
    in_data = in_data.reshape(len(in_data), batch_size, -1)

    ta_data = [sin_wave[idx+1:idx+1+batch_size] for idx in range(count)]
    ta_data = torch.tensor(ta_data, dtype=torch.float)
    ta_data = ta_data.reshape(len(in_data), batch_size, -1)
    return in_data, ta_data, sin_wave

# Training用・Test用データ
train_in, train_ta, train_sin = make_train_data(100, 2, D_SIZE)
test_in, test_ta, test_sin = make_train_data(100, 1, D_SIZE, np.pi / 2)

In [ ]:
# Training用 10番目のデータの可視化
plt.plot(train_in[:, 10, 0], label="train_in")
plt.plot(train_ta[:, 10, 0], label="train_ta")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Test用 20番目のデータの可視化
plt.plot(test_in[:, 20, 0], label="test_in")
plt.plot(test_ta[:, 20, 0], label="test_ta")
plt.legend()
plt.grid()
plt.show()

### STEP5：ネットワーク学習
学習を下記のステップで行う．
- 入力データ・教師データをネットワークに入力
- ネットワークパラメータの微分値をリセット：```optimizer.zero_grad()```
- ネットワークの出力と教師信号の誤差を計算
- 誤差逆伝播：```loss.backward()```
- ネットワークパラメータの更新：```optimizer.step()```
- 以上を繰り返す

In [ ]:
# エポック数：データセットを何周するか
epochs = 100

# 入力データ・教師データをGPUへ
train_in = train_in.to(device)
train_ta = train_ta.to(device)

for epoch in range(epochs):

    # 入力データを順伝播し出力を確認
    output = rnn_bptt1(train_in)

    # ネットワークパラメータの微分値をリセット
    optimizer.zero_grad()

    # ネットワークの出力と教師信号の誤差を計算
    loss = criterion(output, train_ta)

    # 誤差逆伝播
    loss.backward()

    # ネットワークパラメータの更新
    optimizer.step()

    # 誤差のモニタリング
    if epoch % 20 == 0:
        print(f'epoch: {epoch}, loss: {loss.item():.6f}')

In [ ]:
# Training用データの出力を確認
train_ta = train_ta.to("cpu").detach()
output = output.to("cpu").detach()

plt.plot(train_ta[:, 10, 0], label="train_ta (target data)")
plt.plot(output[:, 10, 0], label="inference")

plt.legend()
plt.grid()
plt.show()

In [ ]:
# Test用データに対する出力を確認
test_in = test_in.to(device)
output = rnn_bptt1(test_in)
output = output.to("cpu").detach()

plt.plot(test_ta[:, 20, 0], label="test_ta (target data)")
plt.plot(output[:, 20, 0], label="inference")

plt.legend()
plt.grid()
plt.show()

## 演習
北九州市の気温データをLSTMを用いたRNNで予測するプログラムを作成せよ．

### STEP1：インポート

### STEP2：ネットワーク定義

Note
- 例題とネットワーク構成を変える
- RNN層の代わりにLSTM層を定義する

### STEP3：誤差関数・最適化器

### STEP4：データセット
気象庁からダウンロードした，八幡の平均気温データを用いる．このデータは次の手順で用意したものである．

1. [気象庁|過去の気象データ・ダウンロード](http://www.data.jma.go.jp/gmd/risk/obsdl/index.php) へアクセスする
2. 福岡を選択
3. 八幡を選択して，右△ボタンを押して次へ進む
4. [データの種類]で[日別値]を選び，下の項目から[日平均気温]を選び，次へ
5. [連続した期間で表示する]「2000年1月1日」から「2020年12月31日」にして，次へ
6. 上から「値を表示(格納)する．ただし～」「観測環境などの変化にかかわらず～」「すべて数値で格納」「日付リテラルで格納」を選ぶ
7. 「csvファイルをダウンロード」でデータをダウンロードする

In [ ]:
# データダウンロード
!wget "https://drive.google.com/uc?export=download&id=1MnXcliKiWT9_GKQviuFx20bcNl1O7d0o" -O data.csv
!ls

In [ ]:
# データ読み込み
import pandas as pd
df = pd.read_csv("data.csv", encoding="SHIFT-JIS", header=3)

# "平均気温"カラムのみを取り出す
df = df.iloc[:, 1].values

# 気温の1次元データへ変換する
df = df.reshape(-1, 1)
df = df.astype("float32")

# 入力データを0～1へ正規化する
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))
df_scaled = scaler.fit_transform(df)

# 正規化された結果を確認する
plt.plot(df_scaled)
plt.grid()
plt.show()

In [ ]:
# Training用データとTest用データに分割する
tr_data_size = int(len(df_scaled) * 0.70)
te_data_size = len(df_scaled) - tr_data_size
tr_data = df_scaled[0:tr_data_size]
te_data = df_scaled[tr_data_size:]
print("train size: {}, test size: {} ".format(len(tr_data), len(te_data)))

In [ ]:
# 総データ数
D_SIZE = 365

# 入力データと教師（正解）データの生成
def make_train_data(data_set, batch_size=25):
    count = len(data_set) - batch_size - 1

    in_data = [data_set[idx:idx+batch_size] for idx in range(count)]
    in_data = torch.tensor(in_data, dtype=torch.float)
    in_data = in_data.reshape(len(in_data), batch_size, -1)

    ta_data = [data_set[idx+1:idx+1+batch_size] for idx in range(count)]
    ta_data = torch.tensor(ta_data, dtype=torch.float)
    ta_data = ta_data.reshape(len(ta_data), batch_size, -1)
    return in_data, ta_data, data_set

train_in, train_ta, train_temp = make_train_data(tr_data, D_SIZE)
test_in, test_ta, test_temp = make_train_data(te_data, D_SIZE)

In [ ]:
# Training用 10番目のデータの可視化
plt.plot(train_in[:, 10, 0], label="train_in")
plt.plot(train_ta[:, 10, 0], label="train_ta")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Test用 20番目のデータの可視化
plt.plot(test_in[:, 20, 0], label="test_in")
plt.plot(test_ta[:, 20, 0], label="test_ta")
plt.legend()
plt.grid()
plt.show()

### STEP5：ネットワーク学習